# Выбор параметров модели и кросс-валидация

Главная задача обучаемых алгоритмов – их способность обобщаться, то есть хорошо работать на новых данных. Поскольку, на новых данных мы сразу не можем проверить качество построенной модели (нам ведь надо для них сделать прогноз, то есть истинных значений целевого признака мы для них не знаем), то надо пожертвовать небольшой порцией данных, чтоб на ней проверить качество модели.

Чаще всего это делается одним из 2 способов:

- отложенная выборка (held-out/hold-out set). При таком подходе мы оставляем какую-то долю обучающей выборки (как правило от 20% до 40%), обучаем модель на остальных данных (60-80% исходной выборки) и считаем некоторую метрику качества модели (например, самое простое – долю правильных ответов в задаче классификации) на отложенной выборке.
- кросс-валидация (cross-validation, на русский еще переводят как скользящий или перекрестный контроль). Тут самый частый случай – K-fold кросс-валидация 

![](img/cross_validation.png)

Тут модель обучается K раз на разных (K-1) подвыборках исходной выборки (белый цвет), а проверяется на одной подвыборке (каждый раз на разной, оранжевый цвет). Получаются K оценок качества модели, которые обычно усредняются, выдавая среднюю оценку качества классификации/регресии на кросс-валидации.

Кросс-валидация дает лучшую по сравнению отложенной выборкой оценку качества модели на новых данных. Но кросс-валидация вычислительно дорогостоящая, если данных много.

Кросс-валидация – очень важная техника в машинном обучении (применяемая также в статистике и эконометрике), с ее помощью выбираются гиперпараметры моделей, сравниваются модели между собой, оценивается полезность новых признаков в задаче и т.д. Более подробно можно почитать, например, [тут](https://sebastianraschka.com/blog/2016/model-evaluation-selection-part1.html) у Sebastian Raschka или в любом классическом учебнике по машинному (статистическому) обучению

sklearn документация: http://scikit-learn.org/stable/modules/cross_validation.html

In [2]:
from sklearn import model_selection, datasets

import numpy as np

In [3]:
iris = datasets.load_iris()

In [4]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3)

In [5]:
#убедимся, что тестовая выборка действительно составляет 0.3 от всех данных
float(len(test_labels))/len(iris.data)

0.3

In [7]:
print ('Размер обучающей выборки: {} объектов \nРазмер тестовой выборки: {} объектов'.format(len(train_data),
                                                                                            len(test_data)))

Размер обучающей выборки: 105 объектов 
Размер тестовой выборки: 45 объектов


In [9]:
print ('Обучающая выборка:\n', train_data[:5])
print ('\n')
print ('Тестовая выборка:\n', test_data[:5])

Обучающая выборка:
 [[6.9 3.1 4.9 1.5]
 [5.  3.6 1.4 0.2]
 [7.7 2.6 6.9 2.3]
 [5.7 4.4 1.5 0.4]
 [5.2 2.7 3.9 1.4]]


Тестовая выборка:
 [[6.  2.2 4.  1. ]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.6 0.4]
 [6.8 3.  5.5 2.1]
 [6.5 3.  5.5 1.8]]


In [10]:
print ('Метки классов на обучающей выборке:\n', train_labels)
print ('\n')
print ('Метки классов на тестовой выборке:\n', test_labels)

Метки классов на обучающей выборке:
 [1 0 2 0 1 0 2 1 0 0 2 2 2 1 2 2 2 2 0 2 1 0 2 1 2 0 1 0 0 1 2 2 2 1 1 2 1
 0 2 1 0 1 2 1 1 0 1 0 1 0 1 0 1 2 2 0 1 0 2 2 0 2 1 0 2 0 2 1 1 2 1 0 1 1
 1 1 1 0 0 0 1 2 1 1 0 1 2 2 2 0 2 2 2 0 2 1 0 1 1 1 0 0 2 2 2]


Метки классов на тестовой выборке:
 [1 0 0 2 2 1 0 1 0 0 0 1 2 2 1 0 1 1 2 2 0 2 0 0 0 0 0 2 1 0 1 1 0 1 2 2 2
 1 1 0 2 0 0 0 2]


### Стратегии проведения кросс-валидации

In [19]:
#сгенерируем короткое подобие датасета, где элементы совпадают с порядковым номером
X = range(0,10)

#### KFold

In [13]:
kf = model_selection.KFold(n_splits = 5)
for train_indices, test_indices in kf.split(X):
    print (train_indices, test_indices)

[2 3 4 5 6 7 8 9] [0 1]
[0 1 4 5 6 7 8 9] [2 3]
[0 1 2 3 6 7 8 9] [4 5]
[0 1 2 3 4 5 8 9] [6 7]
[0 1 2 3 4 5 6 7] [8 9]


In [14]:
kf = model_selection.KFold(n_splits = 2, shuffle = True)
for train_indices, test_indices in kf.split(X):
    print (train_indices, test_indices)

[1 3 4 5 8] [0 2 6 7 9]
[0 2 6 7 9] [1 3 4 5 8]


In [15]:
kf = model_selection.KFold(n_splits = 2, shuffle = True, random_state = 1)
for train_indices, test_indices in kf.split(X):
    print (train_indices, test_indices)

[1 3 5 7 8] [0 2 4 6 9]
[0 2 4 6 9] [1 3 5 7 8]


#### StratifiedKFold

In [17]:
y = np.array([0] * 5 + [1] * 5)
print (y)

skf = model_selection.StratifiedKFold(n_splits = 2, shuffle = True, random_state = 0)
for train_indices, test_indices in skf.split(X, y):
    print (train_indices, test_indices)

[0 0 0 0 0 1 1 1 1 1]
[3 4 8 9] [0 1 2 5 6 7]
[0 1 2 5 6 7] [3 4 8 9]


In [18]:
target = np.array([0, 1] * 5)
print (target)

skf = model_selection.StratifiedKFold(n_splits = 2,shuffle = True)
for train_indices, test_indices in skf.split(X, target):
    print (train_indices, test_indices)

[0 1 0 1 0 1 0 1 0 1]
[0 3 4 7] [1 2 5 6 8 9]
[1 2 5 6 8 9] [0 3 4 7]


#### ShuffleSplit

In [20]:
ss = model_selection.ShuffleSplit(n_splits = 10, test_size = 0.2)

for train_indices, test_indices in ss.split(X):
    print (train_indices, test_indices)

[2 7 6 5 8 0 9 4] [1 3]
[3 7 8 6 0 1 9 5] [2 4]
[5 8 1 0 4 9 6 7] [3 2]
[6 1 9 8 2 0 3 4] [5 7]
[4 3 0 6 9 7 5 2] [8 1]
[3 7 4 6 2 8 1 9] [0 5]
[0 4 7 3 1 5 8 9] [6 2]
[6 9 2 0 1 5 3 4] [8 7]
[8 2 6 7 3 5 4 1] [9 0]
[4 0 3 6 1 2 5 9] [7 8]


#### Leave-One-Out

In [21]:
loo = model_selection.LeaveOneOut()

for train_indices, test_index in loo.split(X):
    print (train_indices, test_index)

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]


Больше стратегий проведения кросс-валидации доступно здесь: http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators